In [1]:
import os
import sys
import gensim
from sklearn.decomposition import PCA
from matplotlib import pyplot
import nltk
import math
import time
import re
import numpy as np
from operator import itemgetter
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional

Using TensorFlow backend.


In [2]:
cwd =  os.getcwd()
input1 = open(cwd+'/training/train.txt.text','r')
input2 = open(cwd+'/training/train.txt.labels','r')
temp_train_txt = []
train_labels = []
for sentence in input1:
    sentence = sentence.rstrip()
    temp_train_txt.append(sentence)
for sentence in input2:
    sentence = sentence.rstrip()
    sentence = int(sentence)
    train_labels.append(sentence)
train_labels

[2,
 17,
 0,
 18,
 1,
 9,
 2,
 8,
 13,
 0,
 1,
 2,
 18,
 12,
 3,
 5,
 14,
 16,
 1,
 13,
 16,
 10,
 1,
 0,
 4,
 13,
 9,
 6,
 6,
 6,
 4,
 17,
 2,
 15,
 1,
 2,
 14,
 2,
 0,
 9,
 4,
 0,
 17,
 0,
 0,
 16,
 4,
 2,
 5,
 2,
 0,
 14,
 0,
 4,
 7,
 7,
 3,
 0,
 3,
 5,
 0,
 5,
 4,
 0,
 16,
 0,
 0,
 0,
 14,
 5,
 1,
 8,
 0,
 7,
 9,
 0,
 4,
 12,
 5,
 6,
 3,
 8,
 9,
 6,
 3,
 3,
 11,
 13,
 19,
 16,
 18,
 4,
 2,
 10,
 7,
 11,
 5,
 15,
 17,
 9,
 18,
 1,
 0,
 1,
 0,
 19,
 10,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 7,
 0,
 0,
 2,
 2,
 11,
 18,
 2,
 15,
 0,
 13,
 4,
 0,
 8,
 3,
 0,
 5,
 1,
 1,
 17,
 1,
 1,
 1,
 8,
 7,
 10,
 0,
 4,
 9,
 6,
 0,
 9,
 16,
 1,
 1,
 3,
 9,
 11,
 12,
 10,
 3,
 2,
 8,
 0,
 14,
 18,
 12,
 9,
 3,
 5,
 19,
 11,
 8,
 0,
 6,
 0,
 16,
 5,
 13,
 1,
 2,
 8,
 6,
 12,
 17,
 17,
 2,
 1,
 14,
 0,
 4,
 0,
 2,
 8,
 0,
 3,
 2,
 2,
 0,
 2,
 8,
 2,
 2,
 1,
 0,
 9,
 13,
 2,
 14,
 4,
 1,
 16,
 3,
 4,
 14,
 10,
 6,
 18,
 17,
 16,
 2,
 5,
 17,
 3,
 1,
 0,
 1,
 7,
 2,
 8,
 4,
 13,
 0,
 3,
 17,
 18,
 14,
 3,
 0,
 2

In [3]:
regex_str = [
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)+' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    tokens = [token.lower() for token in tokens]

    html_regex = re.compile('<[^>]+>')
    tokens = [token for token in tokens if not html_regex.match(token)]

    mention_regex = re.compile('(?:@[\w_]+)')
    tokens = ['@user' if mention_regex.match(token) else token for token in tokens]

    url_regex = re.compile('http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+')
    tokens = ['!url' if url_regex.match(token) else token for token in tokens]

    
    return ' '.join([t for t in tokens if t]).replace('rt @user : ','')

In [4]:
temp2_train_txt = []
for line in temp_train_txt:
    line = preprocess(line)
    temp2_train_txt.append(line)


In [5]:
train_txt = []
for sentence in temp2_train_txt:
    sentence = sentence.split()
    for i in range(0,len(sentence)):
#         print(sentence[i],sentence[i][0])
        if len(sentence[i])>1:
            if sentence[i][0] == '#':
                first_word = sentence[i][0]
                sentence[i] = sentence[i].replace('#','')
                sentence.insert(i,first_word)
            
    train_txt.append(sentence)
train_txt

[['lol', '@', 'west', 'covina', ',', 'california'],
 ['things',
  'got',
  'a',
  'little',
  'festive',
  'at',
  'the',
  'office',
  '#',
  'christmas2016',
  '@',
  'redrock',
  '…'],
 ['step',
  'out',
  'and',
  'explore',
  '.',
  '#',
  '️',
  '@',
  'ellis',
  'island',
  'cafe'],
 ['@user', '@', 'cathedral', 'preparatory', 'school'],
 ['my', 'baby', 'bear', '@', "bubby's"],
 ["rupaul's",
  'drag',
  'race',
  'bingo',
  'fun',
  '.',
  'drag',
  'queens',
  'be',
  'sexy',
  '!',
  '#',
  'rupaulsdragrace',
  '@user',
  'abwyman',
  '#',
  'la',
  '…'],
 ['black',
  'history',
  'like',
  'a',
  'mufffffaaaaaka',
  '#',
  'blacchyna',
  'done',
  'thru',
  'her',
  'yugioh',
  'trap',
  'card',
  'like',
  'hell',
  '…'],
 ['@', "bj's", 'restaurant', 'and', 'brewhouse'],
 ['so',
  'lovely',
  'catching',
  'up',
  'with',
  'my',
  'soul',
  'sister',
  '@user',
  '@',
  'university',
  'of',
  'victoria'],
 ['perfect',
  'for',
  'this',
  'weather',
  '️',
  '#',
  'dessert

In [6]:
input3 = open(cwd+'/test/us_test.text','r')
input4 = open(cwd+'/test/us_test.labels','r')
temp_test_txt = []
test_labels = []
for sentence in input3:
    sentence = sentence.rstrip()
    temp_test_txt.append(sentence)
for sentence in input4:
    sentence = sentence.rstrip()
    sentence = int(sentence)
    test_labels.append(sentence)
test_labels

[2,
 10,
 6,
 1,
 16,
 17,
 4,
 10,
 12,
 18,
 0,
 1,
 5,
 7,
 7,
 4,
 0,
 7,
 0,
 10,
 2,
 0,
 0,
 0,
 14,
 3,
 4,
 0,
 6,
 13,
 18,
 18,
 5,
 1,
 11,
 0,
 18,
 13,
 0,
 4,
 0,
 1,
 0,
 4,
 18,
 0,
 16,
 4,
 1,
 1,
 11,
 13,
 2,
 16,
 1,
 0,
 18,
 2,
 2,
 10,
 6,
 6,
 4,
 2,
 1,
 4,
 6,
 18,
 7,
 12,
 10,
 1,
 1,
 0,
 12,
 2,
 1,
 7,
 6,
 7,
 6,
 0,
 0,
 18,
 5,
 17,
 0,
 8,
 7,
 0,
 0,
 0,
 10,
 5,
 0,
 0,
 0,
 0,
 7,
 7,
 0,
 2,
 0,
 1,
 4,
 18,
 17,
 7,
 0,
 14,
 17,
 1,
 3,
 0,
 0,
 13,
 0,
 4,
 14,
 7,
 18,
 0,
 0,
 0,
 6,
 1,
 8,
 2,
 3,
 3,
 6,
 9,
 18,
 11,
 7,
 2,
 5,
 7,
 0,
 18,
 18,
 14,
 6,
 3,
 1,
 15,
 13,
 2,
 8,
 16,
 14,
 14,
 4,
 10,
 1,
 6,
 12,
 0,
 1,
 3,
 1,
 2,
 2,
 11,
 7,
 14,
 0,
 18,
 0,
 9,
 1,
 0,
 7,
 6,
 0,
 2,
 16,
 2,
 1,
 6,
 1,
 10,
 4,
 12,
 16,
 3,
 6,
 8,
 7,
 1,
 3,
 18,
 4,
 7,
 2,
 2,
 19,
 2,
 1,
 4,
 9,
 19,
 5,
 9,
 9,
 1,
 0,
 5,
 8,
 16,
 1,
 17,
 2,
 0,
 3,
 4,
 13,
 18,
 1,
 4,
 2,
 0,
 1,
 1,
 3,
 14,
 9,
 3,
 0,
 4,
 0,
 2,
 15,
 17,


In [7]:
temp2_test_txt = []
for line in temp_test_txt:
    line = preprocess(line)
    temp2_test_txt.append(line)
test_txt = []
for sentence in temp2_test_txt:
    sentence = sentence.split()
    test_txt.append(sentence)
test_txt

[['en', 'pelham', 'parkway'],
 ['the',
  'calm',
  'before',
  '......',
  '|',
  'w',
  '/',
  'sofarsounds',
  '@user',
  '|',
  ':',
  'b',
  '.',
  'hall',
  '.......#sofarsounds…'],
 ['just',
  'witnessed',
  'the',
  'great',
  'solar',
  'eclipse',
  '@',
  'tampa',
  ',',
  'florida'],
 ['this',
  'little',
  'lady',
  'is',
  '26',
  'weeks',
  'pregnant',
  'today',
  '!',
  'excited',
  'for',
  'baby',
  'cam',
  'to',
  'come',
  '!',
  '@',
  'springfield',
  ',…'],
 ['great',
  'road',
  'trip',
  'views',
  '!',
  '@',
  'shartlesville',
  ',',
  'pennsylvania'],
 ['christmas',
  'deals',
  'buy',
  'any',
  '3',
  'small',
  'pomades',
  '1.5',
  'or',
  '1.7',
  'oz',
  'receive',
  'the',
  'f',
  '&amp;s',
  'collector',
  'tin',
  '&amp;',
  'comb',
  '…'],
 ['the',
  '#sisterstunt',
  'was',
  'mad',
  'real',
  'last',
  'night',
  '#miastaxxx',
  '#andreastaxxx',
  '#denverqueen',
  '#staxxxlife',
  '…'],
 ["i'm",
  'starting',
  'to',
  'love',
  'shooting',
  

In [8]:
max_features = 2000
tokenizer = Tokenizer(nb_words=max_features, split=' ')
tokenizer.fit_on_texts(train_txt)

/usr/local/lib/python3.5/dist-packages/keras_preprocessing/text.py:177: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [9]:
n = len(train_txt)
X = tokenizer.texts_to_sequences(train_txt + test_txt)
X_train = X[:n]
X_test = X[n:]
X_train = pad_sequences(X_train, maxlen=41,padding='post')
X_test = pad_sequences(X_test, maxlen=41,padding='post')

In [10]:
X_train[0]

array([154,   2, 225,   5,  77,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=int32)

In [23]:
X_train = np.array(X_train)
X_train.shape

(77360, 41)

In [19]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(categories='auto')
train_labels = np.array(train_labels)
y_train_encoded = enc.fit_transform(train_labels.reshape(-1, 1))
y_train_encoded.shape

(77360, 20)

In [24]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(20,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/home/himansh/venv/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  """
/home/himansh/venv/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, recurrent_dropout=0.2, dropout=0.2)`
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 41, 128)           256000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 20)                3940      
Total params: 514,740
Trainable params: 514,740
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
batch_size = 32
model.fit(X_train, y_train_encoded, nb_epoch = 10, validation_split=0.4, epochs=3)

Train on 46416 samples, validate on 30944 samples
Epoch 1/10
   64/46416 [..............................] - ETA: 1:44 - loss: 1.4906 - acc: 0.5781

/home/himansh/venv/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


46416/46416 [==============================] - 71s 2ms/step - loss: 1.4610 - acc: 0.5438 - val_loss: 2.1191 - val_acc: 0.3988
Epoch 2/10
46416/46416 [==============================] - 70s 2ms/step - loss: 1.4293 - acc: 0.5529 - val_loss: 2.1485 - val_acc: 0.3995
Epoch 3/10
46416/46416 [==============================] - 70s 2ms/step - loss: 1.4085 - acc: 0.5600 - val_loss: 2.1710 - val_acc: 0.3947
Epoch 4/10
46416/46416 [==============================] - 70s 2ms/step - loss: 1.3888 - acc: 0.5656 - val_loss: 2.1966 - val_acc: 0.3935
Epoch 5/10
46416/46416 [==============================] - 70s 2ms/step - loss: 1.3618 - acc: 0.5701 - val_loss: 2.2205 - val_acc: 0.3935
Epoch 6/10
46416/46416 [==============================] - 70s 2ms/step - loss: 1.3339 - acc: 0.5786 - val_loss: 2.2610 - val_acc: 0.3920
Epoch 7/10
46416/46416 [==============================] - 70s 2ms/step - loss: 1.3109 - acc: 0.5843 - val_loss: 2.2880 - val_acc: 0.3873
Epoch 8/10
46416/46416 [============================

In [37]:
predictions = model.predict(X_test)

In [38]:
predictions_decoded = []

for x in predictions:
    for i in range(x.shape[0]):
        if x[i] == max(x):
            predictions_decoded.append(i)
            break

In [39]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
accuracy_score(test_labels, predictions_decoded)

0.35062